# Machine Learning Models
The following classification models are carried out here:
    1. Random Forest
    2. Support Vector Machines
    3. Feed-forward Neural Networks

In [24]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.multioutput
import sklearn.neural_network

%matplotlib inline

# 1. Load in data

In [2]:
X_16 = pd.read_csv('../../../Data/model_inputs/gdf_2016_X.csv')
y_16 = pd.read_csv('../../../Data/model_inputs/gdf_2016_y.csv')
X_17 = pd.read_csv('../../../Data/model_inputs/gdf_2017_X.csv')
y_17 = pd.read_csv('../../../Data/model_inputs/gdf_2017_y.csv')

In [3]:
X_16.head()

,id_trip,mode_f,duration,distance_m,magnitude,carddir_f,start_down,end_downto,weekday,temporal_c,precip,temperatur,startrush,endrush,thrurush,startclust,endclust,land_use_s_f,land_use_e_f
0,1724206,0,460,415.236330,0.227492,0,1,1,1,3,0.000002,28.012522,0,0,0,9,9,0,0
1,1889461,1,447,1843.264582,0.470022,0,1,1,1,5,0.000134,25.844886,1,1,1,0,0,1,1
2,1724219,2,591,2657.421830,0.303495,1,1,1,1,3,0.000240,25.389363,0,0,0,0,9,1,0
3,2071991,3,844,2761.792383,0.223787,2,1,1,1,5,0.001427,24.930720,1,1,1,9,9,0,4
4,1667922,3,1211,1068.301088,0.293601,3,1,1,1,4,0.001429,21.769356,0,0,0,0,0,0,0


In [4]:
y_16.head()

,purpose_f
0,0
1,1
2,0
3,2
4,2


In [5]:
y_16['purpose_f'].value_counts()

0    15554
4    14981
1     7430
2     5790
3     5682
5     2473
6     2262
7     2168
Name: purpose_f, dtype: int64

# 2. Setup model
#### Section Overview:
2.1  Encode model inputs  
2.2  Normalise data and train/test split  
2.3  Data preperation  

### 2.1 Encode model inputs
The model inputs are encoded to improve the performance of models. Specifically by One-hot encoding all the 

#### Encoded inputs:
- Trip mode *(f)*
- Cardinal Direction *(f)*
- Trip starting & ending cluster from K-means (see `./Notebooks/preprocessing/metric_creation/clustering.ipynb`)
- Temporal cluster from LDA
- Land-use at trip starting and ending points *(f)* (see `./Notebooks/preprocessing/metric_creation/land_use_poi.ipynb`)

*(f) == factors*

In [6]:
def encode_model_inputs(data, col):
    encoded_input = ''
    if col in data.columns:
        encoded_input = enc.fit_transform(data[col].values.reshape(-1, 1)).toarray()
    return encoded_input

In [7]:
enc = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')

In [40]:
encoded_inputs = {}
for col in ["mode_f","carddir_f","startclust","endclust", "temporal_c", "land_use_s_f", "land_use_e_f"]:
    encoded_inputs[col] = encode_model_inputs(X_16, col)
    
encoded_inputs['y_codes'] = encode_model_inputs(y_16, 'purpose_f')

In [41]:
for key in encoded_inputs.keys():
    print(encoded_inputs[key].shape)

(56340, 6)
(56340, 16)
(56340, 12)
(56340, 12)
(56340, 5)
(56340, 10)
(56340, 10)
(56340, 8)


In [42]:
encoded_inputs['y_codes']

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

### 2.2 Normalise and split training and testing

In [10]:
def normalise(X):
    X = X / np.amax(X, axis=0)
    return X


def model_setup(X, y, test_size=0.33, norm=False):
    if norm:
        X = normalise(X)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=test_size,\
                                                                                random_state=42, stratify=y)
    return X_train, X_test, y_train, y_test    

In [11]:
feature_list = list(X_16.columns)
feature_list[:3]

['id_trip', 'mode_f', 'duration']

In [12]:
X_train, X_test, y_train, y_test  = model_setup(X_16, y_16, norm=False)

### 2.3 Data preparation

In [13]:
## flatten data i.e. from Pd.Series to np.array
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

# 3. Begin Modelling
#### Section Overview:
3.1 Define model functions and test modelling   
3.2 Preliminary modelling for important feature extraction  
3.3 Re-run models  

#### Model functions:
- `run_rf` == run the random forest classification model.
- `run_sv` == run the support vector machine classification model.
- `run_ann` == run the multi-layer perceptron classification model.  
- `run_mcrf` == *Experimental* run the multi-output random forest classification model.

#### Notes:

#### Technical Notes:
- each model will have a cross-validation option. To use this the function parameters will need to be set to `cv=True` and `cv_val` to the number of k-folds defaulting to `cv_val=5`

### 3.1 Define model functions and test

In [14]:
# All Model functions
def run_rf(X_train, X_test, y_train, y_test, n_estimators=10, cv=False, cv_val=5):
    cv_scores = []
    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1)
    if cv:
        cv_scores = sklearn.model_selection.cross_val_score(clf, X_train, y_train, cv=cv_val)
    clf.fit(X_train, y_train)
    score = clf.score(X_test,y_test)
    preds = clf.predict(X_test)
    print("Random Forest Classifcation accuracy:", score)
    return score, preds, cv_scores


def run_svc(X_train, X_test, y_train, y_test, gamma_val=0.01, C_val=0.1, cv=False, cv_val=5):
    cv_scores = []
    if cv:
        clf = sklearn.svm.SVC(gamma=gamma_val, C=C_val, decision_function_shape='ova')
        cv_scores = sklearn.model_selection.cross_val_score(clf, X_train, y_train, cv=cv_val)
    clf = sklearn.svm.SVC(gamma=gamma_val, C=C_val, decision_function_shape='ova')
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    preds = clf.predict(X_test)  
    print("Support Vector classification accuracy:", score)
    return score, preds, cv_scores


def run_ann(X_train, X_test, y_train, y_test, alpha_val=0.01, C_val=0.1, cv=False, cv_val=5):
    cv_scores = []
    if cv:
        clf = sklearn.neural_network.MLPClassifier(solver='lbfgs', alpha=alpha_val,\
                                            hidden_layer_sizes=(50, 50, 50), random_state=1, max_iter=500)
        cv_scores = sklearn.model_selection.cross_val_score(clf,X_train,y_train, cv=cv_val)
    clf = sklearn.neural_network.MLPClassifier(solver='lbfgs', alpha=alpha_val,\
                                               hidden_layer_sizes=(50, 50, 50), random_state=1, max_iter=500)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    preds = clf.predict(X_test)
    print("Neural Network classification accuracy:", score)
    return score, preds, cv_scores


## Experimental
def run_mcrf(X_train, X_test, y_train, y_test, n_estimators=10, cv=False, cv_val=5):
    cv_scores = []
    clf = sklearn.ensemble.RandomForestClassifier(n_estimators = n_estimators,\
                                                  criterion = 'entropy', random_state = 42)
    multi_target_forest = sklearn.multioutput.MultiOutputClassifier(clf, n_jobs=-1)
    if cv:
        cv_scores = sklearn.model_selection.cross_val_score(multi_target_forest, X_train, y_train, cv=cv_val)
    multi_target_forest.fit(X_train, y_train)
    score = multi_target_forest.score(X_test, y_test)
    preds = multi_target_forest.predict(X_test)
    print("Multi-output Random Forest classification accuracy:", score)
    return score, preds, cv_scores

In [15]:
%%time
run_rf(X_train, X_test, y_train, y_test,cv=False)

Random Forest Classifcation accuracy: 0.4178454256978433
CPU times: user 1.65 s, sys: 113 ms, total: 1.77 s
Wall time: 1.04 s


(0.4178454256978433, array([5, 1, 0, ..., 0, 0, 0]), [])

In [16]:
%%time
run_rf(X_train, X_test, y_train, y_test,cv=True)

Random Forest Classifcation accuracy: 0.42618189641262844
CPU times: user 2.04 s, sys: 667 ms, total: 2.7 s
Wall time: 6.59 s


(0.42618189641262844,
 array([1, 1, 0, ..., 1, 0, 0]),
 array([0.42274593, 0.41128178, 0.41780368, 0.422022  , 0.42340313]))

In [17]:
%%time
run_svc(X_train, X_test, y_train, y_test,cv=False)

Support Vector classification accuracy: 0.27607163986446515
CPU times: user 7min 9s, sys: 11.5 s, total: 7min 20s
Wall time: 8min 55s


(0.27607163986446515, array([0, 0, 0, ..., 0, 0, 0]), [])

In [18]:
%%time
run_svc(X_train, X_test, y_train, y_test,cv=True)

Support Vector classification accuracy: 0.27607163986446515
CPU times: user 23min 44s, sys: 47.8 s, total: 24min 32s
Wall time: 47min 12s


(0.27607163986446515,
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0.27604925, 0.27595339, 0.27606305, 0.27613621, 0.27617281]))

In [25]:
%%time
run_ann(X_train, X_test, y_train, y_test,cv=False)

Neural Network classification accuracy: 0.10084440380788469
CPU times: user 28.6 s, sys: 2.45 s, total: 31 s
Wall time: 22.1 s


(0.10084440380788469, array([3, 3, 3, ..., 3, 3, 3]), [])

In [26]:
%%time
run_ann(X_train, X_test, y_train, y_test,cv=True)

Neural Network classification accuracy: 0.10084440380788469
CPU times: user 9min 15s, sys: 44.7 s, total: 9min 59s
Wall time: 7min 46s


(0.10084440380788469,
 array([3, 3, 3, ..., 3, 3, 3]),
 array([0.19528664, 0.27502648, 0.10941847, 0.16602624, 0.15425391]))

### 3.2 Prepare preliminary RF to subset by feature importance

In [21]:
def calc_feature_imp(clf, feature_list):
    # Get numerical feature importances
    importances = list(clf.feature_importances_)
    # List of tuples with variable and importance
    feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    above_005 = [i[0] if i[1] >= 0.05 else "purpose_f" for i in feature_importances]
    return feature_importances, above_005


def subset_to_imp_features(data, above_005):
    above_005.append("id_trip")
    data = data[data.columns[[col in above_005 for col in data.columns]]]
    return data


def plot_feature_imp(feature_importances, title):    
    fig, ax = plt.subplots(1, figsize=(10,6))
    feat_imp = pd.DataFrame(feature_importances, columns=['importance', 'feature'])
    feat_imp = feat_imp.loc[(feat_imp.importance != 'id_trip')]
    feat_imp.plot(kind='barh', ax=ax, legend=False)
    ax.set_yticklabels([new_column_labels[i] for i in list(feat_imp.importance.values)], size=16);
    plt.xticks(size=16);
    ax.set_xlim(0,0.2)
    ax.axvline(0.05, -10,40, color='r',linestyle='--')
    ax.set_xlabel("Feature Importance", size=20)
    ax.set_ylabel("Feature", size=20)
    ax.set_title("{0}".format(title), size=22)
    return ax

### 3.3 Re-build the models with new subset data

## Model reports

In [22]:
sklearn.metrics.classification_report

<function sklearn.metrics.classification.classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False)>

## Grid Search

In [23]:
sklearn.model_selection.GridSearchCV

sklearn.model_selection._search.GridSearchCV